Imports and connection to database

In [1]:
import psycopg2
from psycopg2 import sql
from dotenv import dotenv_values
import pandas as pd
from datetime import datetime, timedelta
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import Category10
from bokeh.io import curdoc

config = dotenv_values()
DB_HOST = config['DB_HOST']
DB_NAME = config['DB_NAME']
DB_USER = config['DB_USER']
DB_PASSWORD = config['DB_PASSWORD']
DB_PORT = config['DB_PORT']


connection = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=DB_PORT,
        sslmode="require"
    )
print("Connection successful!")

cursor = connection.cursor()

# pd.read_sql("SELECT * FROM ListeningInstances", connection, index_col='id')

Connection successful!


SQL Queries

In [2]:
pd.read_sql("SELECT MAX(id) FROM ListeningInstances", connection)

/tmp/ipykernel_85265/4106069600.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT MAX(id) FROM ListeningInstances", connection)


,max
0,24882


In [3]:

days_back = 14


stamp_back = datetime.now() - timedelta(days=days_back)

stamp_back = datetime.isoformat(stamp_back)

cumulative_listens_by_song = f'''
SELECT 
    timestamp, song_fk, songs.name,
    SUM(1) OVER (
        PARTITION BY song_fk 
        ORDER BY timestamp 
        RANGE BETWEEN INTERVAL '{days_back}' DAY PRECEDING AND CURRENT ROW
    ) AS cumulative_listens
FROM ListeningInstances
JOIN songs ON song_fk=song_id
JOIN artists ON artist_id=artists_fk
WHERE user_fk = 'jpbroz'
ORDER BY cumulative_listens DESC;
'''

cumulative_listens_by_artist = f'''
SELECT 
    timestamp, song_fk, artists.genres,
    SUM(duration) OVER (
        PARTITION BY genres
        ORDER BY timestamp 
        RANGE BETWEEN INTERVAL '{days_back}' DAY PRECEDING AND CURRENT ROW
    ) AS cumulative_listens, COUNT(song_fk) AS total_listens
FROM ListeningInstances
JOIN songs ON song_fk=song_id
JOIN artists ON artist_id=artists_fk
WHERE user_fk = 'jpbroz' AND genres!= '{{}}' AND timestamp >= '{stamp_back}'
ORDER BY cumulative_listens DESC;
'''
test = f'''
SELECT 
    timestamp, 
    song_fk, 
    artists.genres,
    SUM(duration) OVER (
        PARTITION BY genres
        ORDER BY timestamp 
        RANGE BETWEEN INTERVAL '{days_back}' DAY PRECEDING AND CURRENT ROW
    ) AS cumulative_listens,
    COUNT(song_fk) OVER (
        PARTITION BY genres
    ) AS total_listens
FROM ListeningInstances
JOIN songs ON song_fk = song_id
JOIN artists ON artist_id = artists_fk
WHERE user_fk = 'jpbroz' AND genres != '{{}}' AND timestamp >= '{stamp_back}'
ORDER BY cumulative_listens DESC;
'''

df = pd.read_sql(test, connection)
df

/tmp/ipykernel_85265/3780049771.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(test, connection)


,timestamp,song_fk,genres,cumulative_listens,total_listens
0,2025-08-22 02:42:03.931000+00:00,57CWAcG4JyxR1nTuBsRW4a,unspecified,279.24,78
1,2025-08-21 05:11:55.050000+00:00,2L4HF8iN9aH2FH7jEs7EV6,unspecified,274.38,78
2,2025-08-21 04:14:04.119000+00:00,0u5aRUEwlWhtO32VASKoLi,unspecified,269.52,78
3,2025-08-21 01:24:49.155000+00:00,3hxIUxnT27p5WcmjGUXNwx,unspecified,266.79,78
4,2025-08-21 01:21:49.388000+00:00,57CWAcG4JyxR1nTuBsRW4a,unspecified,264.05,78
...,...,...,...,...,...
965,2025-08-17 06:00:44.758000+00:00,3QVe9gr87GYcRXAkVpPVYl,{weirdcore},1.58,5
966,2025-08-16 14:20:26.661000+00:00,46A9awz7xtpDGNeNtaqWMh,nantes indie,1.45,9
967,2025-08-11 21:25:00.432000+00:00,4LAa9a5VMkk3mN8BHhB93c,{jazz},1.31,1
968,2025-08-21 02:01:37.037000+00:00,5omLpU1N0uN51OSXL62VAM,hypnagogic pop,1.16,1


Class Definition

In [4]:
class Insights():
    def __init__(self, connection, username):
        self.connection = connection
        self.user = username
        output_notebook()
    def plot_bokeh(self, df, y_col, title, filter_on_id, filter_on,  y_label, k, days_back):
        if k > 10:
            k = 10
        
        k=5
        
        top_k_ids = df[filter_on_id].value_counts().head(k).index
        colors = Category10[len(top_k_ids)]
        top_k = df[filter_on].value_counts().head(k).index
        filtered = df[df[filter_on_id].isin(top_k_ids)]

        scaler = 20*k if k>5 else 0

        p = figure(
            title=title,
            x_axis_label="Date",
            y_axis_label=y_label,
            x_axis_type="datetime",
            width=1000 + scaler,
            height=600 + scaler
            )

        p.y_range.bounds = (min(df[y_col]), max(df[y_col])+5)
        p.x_range.bounds = (datetime.now() - timedelta(days_back), datetime.now() + timedelta(days=0.5))

        i=0
        for id, field in zip(top_k_ids, top_k):
            song_data = filtered[filtered[filter_on_id] == id]
            p.line(
                x=song_data['timestamp'],
                y=song_data[y_col],
                legend_label=field,
                line_width=8,
                color = colors[i],
                line_cap = 'round'
            )
            i+=1

        p.legend.location="top_left"
        p.legend.glyph_height=5
        curdoc().theme = 'dark_minimal'
        show(p)
        return p

    def cumulative_listens_by_artist(self, days_back):
        stamp_back = datetime.now() - timedelta(days=days_back)

        stamp_back = datetime.isoformat(stamp_back)

        sql = f'''
        SELECT 
            timestamp, artist_id, artists.name,
            SUM(1) OVER (
                PARTITION BY artist_id 
                ORDER BY timestamp 
                RANGE BETWEEN INTERVAL '{days_back}' DAY PRECEDING AND CURRENT ROW
            ) AS cumulative_listens
        FROM ListeningInstances
        JOIN songs ON song_fk=song_id
        JOIN artists ON artist_id=artists_fk
        WHERE user_fk = '{self.user}' AND timestamp >= '{stamp_back}'
        ORDER BY cumulative_listens DESC;
        '''
        df = pd.read_sql(sql, self.connection)
        self.plot_bokeh(df, 'cumulative_listens', f'Cumulative listens by artist over the last {days_back} days', 'artist_id', 'name', '# of Listens', 5, days_back)

        # return df 

    def cumulative_listens_by_song(self, days_back):
        stamp_back = datetime.now() - timedelta(days=days_back)

        stamp_back = datetime.isoformat(stamp_back)

        sql = f'''
            SELECT 
                timestamp, song_fk, songs.name,
                SUM(1) OVER (
                    PARTITION BY song_fk 
                    ORDER BY timestamp 
                    RANGE BETWEEN INTERVAL '{days_back}' DAY PRECEDING AND CURRENT ROW
                ) AS cumulative_listens
            FROM ListeningInstances
            JOIN songs ON song_fk=song_id
            JOIN artists ON artist_id=artists_fk
            WHERE user_fk = '{self.user}' AND timestamp >= '{stamp_back}'
            ORDER BY cumulative_listens DESC;
            '''
        df = pd.read_sql(sql, self.connection)
        fig = self.plot_bokeh(df, 'cumulative_listens', f'Cumulative listens by song over the last {days_back} days', 'song_fk', 'name', '# of Listens', 5, days_back)
        print(type(fig))
        return fig
        # return df 
    
    def cumulative_listens_by_genre(self, days_back):
        stamp_back = datetime.now() - timedelta(days=days_back)

        stamp_back = datetime.isoformat(stamp_back)

        sql = f'''
            SELECT 
                timestamp, song_fk, artists.genres,
                SUM(1) OVER (
                    PARTITION BY genres
                    ORDER BY timestamp 
                    RANGE BETWEEN INTERVAL '{days_back}' DAY PRECEDING AND CURRENT ROW
                ) AS cumulative_listens
            FROM ListeningInstances
            JOIN songs ON song_fk=song_id
            JOIN artists ON artist_id=artists_fk
            WHERE user_fk = '{self.user}' AND genres!= '{{}}' AND timestamp >= '{stamp_back}' AND genres!='unspecified'
            ORDER BY cumulative_listens DESC;
            '''
        
        df = pd.read_sql(sql, self.connection)
        df['genres'] = df['genres'].apply(lambda x : x.strip('{}"'))
        self.plot_bokeh(df, 'cumulative_listens', f'Cumulative listens by genre over the last {days_back} days', 'song_fk', 'genres', '# of Listens', 5, days_back)
        # return df 

    def cumulative_minutes_by_genre(self, days_back):
        stamp_back = datetime.now() - timedelta(days=days_back)

        stamp_back = datetime.isoformat(stamp_back)

        sql = f'''
            SELECT 
                timestamp, song_fk, artists.genres,
                SUM(duration) OVER (
                    PARTITION BY genres
                    ORDER BY timestamp 
                    RANGE BETWEEN INTERVAL '{days_back}' DAY PRECEDING AND CURRENT ROW
                ) AS cumulative_minutes
            FROM ListeningInstances
            JOIN songs ON song_fk=song_id
            JOIN artists ON artist_id=artists_fk
            WHERE user_fk = '{self.user}' AND timestamp >= '{stamp_back}' AND genres!='unspecified'
            ORDER BY cumulative_minutes DESC;
            '''
        
        df = pd.read_sql(sql, self.connection)
        df['genres'] = df['genres'].apply(lambda x : x.strip('{}"'))
        df = df[df['genres']!= '']
        self.plot_bokeh(df, 'cumulative_minutes', f'Cumulative minutes listened by genre over the last {days_back} days', 'song_fk', 'genres', 'Minutes', 5, days_back)

        # return df     
    

    def cumulative_minutes_by_song(self, days_back):
        stamp_back = datetime.now() - timedelta(days=days_back)

        stamp_back = datetime.isoformat(stamp_back)
        sql = f'''
            SELECT 
                timestamp, song_fk, songs.name,
                SUM(duration) OVER (
                    PARTITION BY song_fk 
                    ORDER BY timestamp 
                    RANGE BETWEEN INTERVAL '{days_back}' DAY PRECEDING AND CURRENT ROW
                ) AS cumulative_minutes
            FROM ListeningInstances
            JOIN songs ON song_fk=song_id
            JOIN artists ON artist_id=artists_fk
            WHERE user_fk = '{self.user}' AND timestamp >= '{stamp_back}'
            ORDER BY cumulative_minutes DESC;
            '''
        df = pd.read_sql(sql, self.connection)
        self.plot_bokeh(df, 'cumulative_minutes', f'Cumulative minutes listened by song over the last {days_back} days', 'song_fk', 'name', 'Minutes', 5, days_back)
        # return df    
        
    def cumulative_minutes_by_artist(self, days_back):
        stamp_back = datetime.now() - timedelta(days=days_back)

        stamp_back = datetime.isoformat(stamp_back)
        sql = f'''
        SELECT 
            timestamp, artist_id, artists.name,
            SUM(duration) OVER (
                PARTITION BY artist_id 
                ORDER BY timestamp 
                RANGE BETWEEN INTERVAL '{days_back}' DAY PRECEDING AND CURRENT ROW
            ) AS cumulative_minutes
        FROM ListeningInstances
        JOIN songs ON song_fk=song_id
        JOIN artists ON artist_id=artists_fk
        WHERE user_fk = '{self.user}' AND timestamp >= '{stamp_back}'
        ORDER BY cumulative_minutes DESC;
        '''
        df = pd.read_sql(sql, self.connection)
        self.plot_bokeh(df, 'cumulative_minutes', f'Cumulative minutes listened by artist over the last {days_back} days', 'artist_id', 'name', 'Minutes', 5, days_back)

        # return df        

    def total_listens_by_artist(self, days_back):
        stamp_back = datetime.now() - timedelta(days=days_back)

        stamp_back = datetime.isoformat(stamp_back)
        sql = f'''
        SELECT 
            timestamp, artist_id, artists.name,
            COUNT(*) OVER (
                PARTITION BY artist_id 
                ORDER BY timestamp 
                RANGE BETWEEN INTERVAL '{days_back}' DAY PRECEDING AND CURRENT ROW
            ) AS total_listens
        FROM ListeningInstances
        JOIN songs ON song_fk=song_id
        JOIN artists ON artist_id=artists_fk
        WHERE user_fk = '{self.user}' AND timestamp >= '{stamp_back}'
        ORDER BY total_listens DESC;
        '''
        df = pd.read_sql(sql, self.connection)
        return df
    


In [5]:
# connection.rollback()
sql = "DELETE FROM Users WHERE username='andrewbroz';"
cursor.execute(sql)
connection.commit()
sql='SELECT * FROM Users;'
pd.read_sql(sql, connection)

/tmp/ipykernel_85265/3757476364.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, connection)


,username,password,oauth_refresh_token,total_listening_minutes
0,iheartjp,iheartjppassword,AQDfnfXJUNdMGj-pc877U4mydbRdO_xwsQeEZrBZwXCczb...,62910.47
1,jpbroz,hellovro,AQC4s2INtLv1NgXjIX8MoY95UJfPINlmMag_Q1uC-gGQSO...,90307.92
2,bodhi_monster,Soccerisfun!1,AQD5z8JHSE-FKwdRpNxpw47hm666n2eSRIsUTZu7WKEiyG...,60153.57
3,stevenbroz,jpspotifyproject,AQAeliB4kvsUe7V6o_hjsMAsAus2NQAT18Blcd1jb9xWbO...,60955.16


Save all to csv

In [7]:
days_back = 30

insights = Insights(connection, 'bodhi_monster')
# output_notebook()
#Work
insights.cumulative_listens_by_song(days_back)
insights.cumulative_listens_by_genre(days_back)
# insights.cumulative_minutes_by_song(days_back)
# insights.cumulative_minutes_by_artist(days_back)
insights.cumulative_listens_by_artist(days_back)
# insights.cumulative_minutes_by_genre(days_back)
#Fix

# insights.plot_bokeh(df, 'cumulative_listens', f'Cumulative listens by song over the last {days_back} days', 'song_fk', 'name', 'Cumulative listens', 5, days_back)




Loading BokehJS ...

C:\Users\jpotr\AppData\Local\Temp\ipykernel_16396\3873170367.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.connection)


<class 'bokeh.plotting._figure.figure'>


C:\Users\jpotr\AppData\Local\Temp\ipykernel_16396\3873170367.py:119: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.connection)


C:\Users\jpotr\AppData\Local\Temp\ipykernel_16396\3873170367.py:69: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.connection)


In [7]:
from bokeh.embed import json_item
import json
t = insights.cumulative_listens_by_song(days_back)
# (t, filename='example_plot.png')
# print(type(t))
temp = json.dumps(json_item(t, 'example_plot'))

with open('example_plot.json', 'w') as writer:
    writer.write(temp)

C:\Users\jpotr\AppData\Local\Temp\ipykernel_13968\71363479.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.connection)


<class 'bokeh.plotting._figure.figure'>
<class 'bokeh.plotting._figure.figure'>


In [15]:
# with open('example_plot.json', 'w') as writer:
#     writer.write(temp)

import bokeh
print(bokeh.__version__)

3.6.0
